# PawCore Intelligence Analysis: Solving the Regional Revenue Mystery

## Executive Business Intelligence with Snowflake AI Functions
This notebook demonstrates how to use Snowflake's AI SQL functions to transform a complex business crisis into actionable strategic intelligence. **The Challenge**: One region experienced a 40% revenue drop in Q4 2024 while other regions remained stable.

**Data Sources:**
- **Structured Data**: Device telemetry, quality logs, customer reviews, internal communications
- **Unstructured Data**: Quality control standards, technical documentation, multimedia content
- **Business Context**: 199 dollar device price, 450 dollar customer lifetime value, 3 global markets across different climates

# Strategic AI SQL Functions Analysis

## 1. Quality Gap Discovery (AI_EXTRACT)
**Business Purpose**: Uncover systematic quality control failures in documentation  
**Strategic Use Case**: Identify why products pass QC but fail in real-world conditions  
**Intelligence Focus**: Environmental testing gaps that correlate with regional climate differences  
**Executive Output**: Precise identification of "optional humidity testing" as systematic failure point

## 2. Customer Impact Quantification (AI_SENTIMENT)
**Business Purpose**: Transform customer feedback into measurable business impact  
**Strategic Use Case**: Correlate sentiment patterns with specific product lots and regions  
**Intelligence Focus**: Multi-dimensional sentiment analysis revealing 90% negative sentiment in problem region  
**Executive Output**: Clear connection between LOT341, humidity failures, and customer dissatisfaction

## 3. Issue Categorization & Pattern Recognition (AI_CLASSIFY)
**Business Purpose**: Automatically categorize customer complaints into specific problem types  
**Strategic Use Case**: Identify dominant failure patterns across negative reviews  
**Intelligence Focus**: Classification reveals moisture problems as the primary issue category in EMEA  
**Executive Output**: Systematic identification of moisture-related failures as the root cause of negative sentiment

## 4. Financial Impact Modeling (AI_COMPLETE)
**Business Purpose**: Generate board-ready financial impact analysis and strategic recommendations  
**Strategic Use Case**: Convert customer sentiment metrics into precise revenue impact predictions  
**Intelligence Focus**: Region-specific financial modeling with immediate action plans  
**Executive Output**: 130K dollar crisis quantification with 418K dollar growth opportunity identification

In [ ]:
-- Set up our environment
USE ROLE accountadmin;
USE WAREHOUSE PAWCORE_DEMO_WH;
USE DATABASE PAWCORE_ANALYTICS;
USE SCHEMA SEMANTIC;

In [ ]:
-- Telemetry data (random sample)
SELECT 'TELEMETRY' as source, device_id, lot_number, battery_level, humidity_reading, region, timestamp::date as date
FROM DEVICE_DATA.TELEMETRY 
ORDER BY RANDOM()
LIMIT 5;

In [ ]:
-- Slack messages (random sample)
SELECT 'SLACK_MESSAGES' as source, slack_channel, user_name, text, thread_id
FROM SUPPORT.SLACK_MESSAGES 
ORDER BY RANDOM()
LIMIT 10;

In [ ]:
-- Random sample of parsed document content
SELECT 
    relative_path,
    file_name,
    LENGTH(content) as content_length,
    LEFT(content, 200) || '...' as content_preview
FROM UNSTRUCTURED.PARSED_CONTENT
ORDER BY RANDOM()
LIMIT 5;

In [ ]:
# Resume Image: Technical Wrong Focus Resume
import warnings, base64, mimetypes, streamlit as st
from snowflake.snowpark.context import get_active_session

warnings.filterwarnings("ignore")
session = get_active_session()

try:
    # Read resume image directly from stage
    stage_path = "@PAWCORE_ANALYTICS.SEMANTIC.PAWCORE_DATA_STAGE/HR/resume_technical_wrong_focus-1.png"
    img_bytes = session.file.get_stream(stage_path, decompress=False).read()
    
    # Guess MIME type from extension
    mime, _ = mimetypes.guess_type("resume_technical_wrong_focus-1.png")
    mime = mime or "image/png"
    
    # Convert to base64 and display
    b64 = base64.b64encode(img_bytes).decode("utf-8")
    st.title("Resume Example")
    st.markdown(f'<img src="data:{mime};base64,{b64}" alt="resume_technical_wrong_focus" style="max-width:800px; border: 1px solid #ddd;">',
                unsafe_allow_html=True)
    st.caption("PawCore HR: Resume")
    
except Exception as e:
    st.write(f"Could not load resume image: {str(e)}")
    st.write("Checking available resume images...")
    
    # Fallback: check what resume images exist
    check_result = session.sql("""
        SELECT relative_path 
        FROM DIRECTORY(@PAWCORE_ANALYTICS.SEMANTIC.PAWCORE_DATA_STAGE) 
        WHERE relative_path ILIKE '%resume%' 
        AND (relative_path ILIKE '%.png' OR relative_path ILIKE '%.jpg' OR relative_path ILIKE '%.jpeg')
    """).collect()
    
    if check_result:
        st.write("Available resume images:")
        for img in check_result:
            st.write(f"- {img['RELATIVE_PATH']}")
    else:
        st.write("No resume images found in stage")

In [ ]:
# Product Image: Barkour.jpg
import warnings, base64, mimetypes, streamlit as st
from snowflake.snowpark.context import get_active_session

warnings.filterwarnings("ignore")
session = get_active_session()

try:
    # Read image directly from stage
    stage_path = "@PAWCORE_ANALYTICS.SEMANTIC.PAWCORE_DATA_STAGE/images/barkour.jpg"
    img_bytes = session.file.get_stream(stage_path, decompress=False).read()
    
    # Guess MIME type from extension (fallback to jpeg)
    mime, _ = mimetypes.guess_type("barkour.jpg")
    mime = mime or "image/jpeg"
    
    # Convert to base64 and display
    b64 = base64.b64encode(img_bytes).decode("utf-8")
    st.title("")
    st.markdown(f'<img src="data:{mime};base64,{b64}" alt="barkour.jpg" style="max-width:800px;">',
                unsafe_allow_html=True)
    st.caption("")
    
except Exception as e:
    st.write(f"Could not load image: {str(e)}")
    st.write("Checking if image exists in stage...")
    
    # Fallback: check if file exists
    check_result = session.sql("""
        SELECT relative_path 
        FROM DIRECTORY(@PAWCORE_ANALYTICS.SEMANTIC.PAWCORE_DATA_STAGE) 
        WHERE relative_path ILIKE '%barkour%'
    """).collect()
    
    if check_result:
        st.write(f"Found image at: {check_result[0]['RELATIVE_PATH']}")
    else:
        st.write("No barkour image found in stage")

## **How AI_EXTRACT Works**

**AI_EXTRACT** is like having a smart assistant read through documents and pull out specific information you need:

- **Input**: Unstructured text (documents, reports, emails) + list of fields to extract
- **Process**: Uses AI to understand context and identify relevant information
- **Output**: Structured JSON with the exact data points you requested
- **Business Value**: Transforms messy documents into clean, queryable data

**Example**: Give it a QC report and ask for `['testing_protocols', 'identified_gaps']` → Get structured data about testing procedures and quality issues!

In [ ]:
-- EXECUTIVE QUESTION: "EMEA revenue dropped significantly in Q4 - what quality issues could explain this?"
--
-- BUSINESS CONTEXT: EMEA region showing major revenue decline while other regions stable.
-- Need to investigate if manufacturing/QC gaps could cause region-specific failures.
--
-- AI_EXTRACT GOAL: Find quality control testing gaps that could explain why products
-- fail specifically in EMEA's environmental conditions (high humidity climate).
SELECT AI_EXTRACT(
  file => TO_FILE('@PAWCORE_ANALYTICS.SEMANTIC.PAWCORE_DATA_STAGE/Document_Stage/QC_standards_SEPT24.pdf'),
  responseFormat => [['water_resistance_tests', 'List: What water resistance tests are performed?'],
  ['humidity_testing', 'List: How is humidity testing documented or required?'],
  ['temperature_tests', 'List: What temperature testing procedures are performed?']]
);

## Results: AI_EXTRACT Analysis

## What We Discovered:

**Temperature Testing:** 3 comprehensive procedures found
- Cold Test (-10°C for 4 hours)
- Heat Test (45°C for 4 hours)  
- Temperature Cycle Test (5 cycles)

**Water Resistance Testing:** 4 rigorous tests implemented
- Light Rain Simulation (IPX4)
- Water Jet Test (IPX5)
- Temporary Immersion (IPX7)
- Basic Splash Test

**Humidity Testing:** CONDITIONAL ONLY - "Humidity (if available)"
- Not a required/standard protocol
- Only tested when equipment happens to be available
- No dedicated humidity validation procedures

**Critical Discovery:** Humidity testing is optional, not mandatory in QC protocols

## Revenue Mystery Connection

**Found the smoking gun!** 

AI_EXTRACT revealed that while PawCore has rigorous temperature and water resistance protocols, humidity testing is only performed "if available" - meaning it's NOT a required part of quality control. This explains why devices pass QC but fail catastrophically in EMEA's humid climate (65-75% humidity), directly causing the regional revenue collapse!



## **How AI_SENTIMENT Works**

**AI_SENTIMENT** is like having an expert analyst read customer feedback and tell you exactly how people feel:

- **Input**: Text (reviews, comments, social media)
- **Process**: AI analyzes emotional tone and context to determine sentiment
- **Output**: Sentiment labels (positive, negative, neutral, mixed, unknown)
- **Business Value**: Instantly understand customer satisfaction patterns and identify problem areas

**This Query**: Analyzes a random sample of 20 reviews across all regions to compare sentiment patterns. By examining reviews from Americas, APAC, and EMEA side-by-side, we can quickly identify which regions have predominantly negative sentiment versus positive/neutral - revealing that EMEA shows a dramatically different sentiment pattern than other regions!


In [ ]:
-- Sentiment analysis using AI_SENTIMENT - Random sample of 20 reviews across all regions
SELECT
    REGION,
    DATE,
    AI_SENTIMENT(REVIEW_TEXT):categories[0]:sentiment::STRING as overall_sentiment,
    REVIEW_TEXT
FROM SUPPORT.CUSTOMER_REVIEWS
WHERE DATE >= '2024-10-01' 
AND DATE <= '2024-12-31'
ORDER BY RANDOM()
LIMIT 20;

## **How AI_CLASSIFY Works**

**AI_CLASSIFY** automatically categorizes text into predefined labels, like having an expert triage specialist sort issues instantly:

- **Input**: Text + list of category labels to classify into
- **Process**: AI reads the content and determines which category best fits
- **Output**: The most relevant category label(s) from your predefined list
- **Business Value**: Automatically organize and route issues without manual review

**This Query**: Takes EMEA's negative reviews and classifies each into problem types (`battery_issues`, `moisture_problems`, `build_quality`, etc.) to quickly identify the dominant failure pattern - revealing that moisture problems are the primary issue driving negative sentiment!


In [ ]:
-- Classify negative reviews into problem categories using AI_CLASSIFY
SELECT
    LOT_NUMBER,
    DATE,
    AI_CLASSIFY(REVIEW_TEXT, ['battery_issues', 'moisture_problems', 'build_quality', 'performance_issues', 'durability_concerns']):labels[0]::STRING as problem_category,
    AI_SENTIMENT(REVIEW_TEXT):categories[0]:sentiment::STRING as overall_sentiment,
    AI_EXTRACT(REVIEW_TEXT, ['specific_issues', 'root_cause', 'failure_mode']) as extracted_details,
    DEVICE_ID,
    LEFT(REVIEW_TEXT, 150) || '...' as review_preview
FROM SUPPORT.CUSTOMER_REVIEWS
WHERE DATE >= '2024-10-01' 
AND DATE <= '2024-12-31'
AND REGION = 'EMEA'
AND AI_SENTIMENT(REVIEW_TEXT):categories[0]:sentiment::STRING = 'negative'
ORDER BY DATE;

## **Results: AI_SENTIMENT/AI_CLASSIFY Analysis**

**What We Discovered:**
- **Regional Sentiment Patterns**: Clear differences in customer sentiment across regions
  - **EMEA**: Predominantly negative sentiment in Q4 2024 reviews
  - **Americas & APAC**: Mix of positive and neutral sentiment
- **EMEA-Specific Issues**: Negative reviews consistently mention:
  - Device failures in humid/wet conditions
  - "Stops working after getting wet"
  - "Fails constantly in humidity"
  - Performance degradation in European climate
- **Manufacturing Lot Connection**: **LOT 341** consistently appears in EMEA negative reviews
- **Timeline Correlation**: EMEA negative sentiment concentrated in Q4 2024 (Oct-Dec)
- **Mystery Clue**: Regional sentiment divide + specific lot number suggests manufacturing defect affecting humidity resistance

**Revenue Mystery Connection**: AI_SENTIMENT reveals a clear regional pattern - EMEA shows overwhelmingly negative sentiment while other regions remain positive, with **LOT 341** as the common thread. This points to a manufacturing defect in LOT 341 that fails specifically in humid environments, directly causing the regional revenue collapse!

## **How AI_COMPLETE Works**

**AI_COMPLETE** generates intelligent responses to prompts - like having ChatGPT directly in your SQL queries:

- **Input**: Model name + prompt (can ask questions, request analysis, generate text, summarize, etc.)
- **Process**: Advanced AI reasoning powered by leading LLM models (Mistral, Llama, etc.)
- **Output**: Free-form text responses tailored to your prompt
- **Business Value**: Unlimited possibilities - text generation, analysis, recommendations, predictions, and more

**This Query**: Uses AI_COMPLETE for **financial analysis and predictions** by feeding it customer review metrics (ratings, review counts, region, lot number) along with business context. The AI analyzes these metrics and generates revenue impact predictions plus strategic recommendations - transforming customer sentiment data into quantified financial insights and actionable business plans!

In [ ]:
-- AI_COMPLETE: Predict Financial Impact from Customer Sentiment
-- Demonstrates AI's ability to connect sentiment data to business outcomes

WITH customer_metrics AS (
    SELECT 
        region,
        lot_number,
        ROUND(AVG(rating), 2) as avg_rating,
        COUNT(*) as review_count
    FROM PAWCORE_ANALYTICS.SUPPORT.CUSTOMER_REVIEWS
    WHERE date >= '2024-10-01'
    GROUP BY region, lot_number
)
SELECT 
    region,
    lot_number,
    avg_rating,
    review_count,
AI_COMPLETE(
    'mistral-large2',
    'Customer rating ' || avg_rating || '/5 with ' || review_count || 
    ' reviews. Estimate monthly revenue impact as percentage. Response format: "X% revenue impact"'
) as revenue_impact
FROM customer_metrics
ORDER BY avg_rating;

## **Results: AI_COMPLETE Analysis**

**What We Discovered:**
- **Customer Satisfaction Impact**: 2.67/5 average rating from 1,071 reviews
- **Revenue Potential**: Operating at only 53.4% of potential revenue capacity
- **Financial Impact**: Estimated 46.6% revenue impact due to poor customer satisfaction
- **Scale of Issue**: Large review volume (1,071) indicates widespread customer experience problems
- **Business Reality**: Low ratings directly correlate to significant revenue loss

**Revenue Mystery Connection**: AI_COMPLETE quantifies the customer satisfaction crisis - with ratings this low across over 1,000 reviews, we're losing nearly half our potential revenue due to poor customer experience!

## **Mystery Solved: Complete AI-Driven Business Intelligence**

**The Revenue Mystery Revealed Through Strategic AI Analysis:**

Using Snowflake's AI SQL functions, we transformed raw data into executive-level business intelligence:

1. **AI_EXTRACT** → Uncovered critical QC gap: humidity testing documented as "if available" (optional/conditional), not a required quality standard - creating systematic failure mode
2. **AI_SENTIMENT** → Revealed clear regional sentiment divide: EMEA predominantly negative vs Americas/APAC positive, with EMEA reviews consistently mentioning humidity/moisture failures
3. **AI_CLASSIFY** → Systematically categorized negative reviews, identifying moisture problems as the dominant failure category and pinpointing **LOT 341** as the problematic manufacturing batch
4. **AI_COMPLETE** → Generated precise financial impact predictions: 129,800 EMEA revenue loss vs 418K opportunity in high-performing regions

**Strategic Business Intelligence Delivered:**
- **Root Cause Identification**: Quality control gap (optional humidity testing) + manufacturing defect (LOT 341) → systematic product failures in EMEA's humid climate (65-75% humidity)
- **Regional Pattern Analysis**: EMEA shows overwhelmingly negative sentiment while Americas/APAC remain positive - proving environment-specific failures
- **Manufacturing Lot Correlation**: LOT 341 consistently appears in negative EMEA reviews, linking specific production batch to regional failures
- **Financial Impact Quantification**: 129,800 EMEA revenue crisis with specific recovery recommendations + 418K growth opportunity identification
- **Executive Recommendations**: Board-ready action plans for crisis management (LOT 341 recall/investigation), QC protocol updates (mandatory humidity testing), and growth optimization

**Transformational AI Value:**
- **Speed**: Minutes vs. weeks to connect QC documentation gaps → customer complaints → specific manufacturing lots → precise revenue impact
- **Depth**: Multi-dimensional analysis spanning unstructured documents, customer sentiment, problem categorization, and financial modeling
- **Precision**: Specific root causes (optional humidity testing + LOT 341 defect), exact failure patterns (moisture problems), and quantified financial impact (130K crisis, 418K opportunity)
- **Strategic Impact**: Executive-level business intelligence that delivers complete end-to-end analysis from quality documentation to actionable recommendations

**Key Takeaway**: Snowflake's AI SQL functions don't just analyze data—they generate strategic business intelligence that connects quality control documents, customer reviews, manufacturing data, and financial metrics into a complete narrative with precise root causes, specific problematic lots, and actionable executive recommendations! 
